In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from pathlib import Path

data_path = Path.cwd() / "smart_meters_london_2013.csv"
df = pd.read_csv(data_path, parse_dates=["timestamp"])

timestamps = df[["timestamp"]]
data = df.drop(columns=["timestamp"]).values

def normalize(data):
    mean = np.mean(data, axis=0, keepdims=True)
    std = np.std(data, axis=0, keepdims=True)
    normalized_data = (data - mean) / (std + 1e-8)
    return normalized_data, mean, std

data_norm, mean, std = normalize(data)

# pytorch
class EnergyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx]

dataset = EnergyDataset(data_norm)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()

        # encoder
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2_mu = nn.Linear(128, latent_dim)
        self.fc2_logvar = nn.Linear(128, latent_dim)

        # decoder
        self.fc3 = nn.Linear(latent_dim, 128)
        self.fc4 = nn.Linear(128, input_dim)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc2_mu(h)
        logvar = self.fc2_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std  # statt normaler Verteilung nehmen wir reale Werte!

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        return torch.tanh(self.fc4(h))  # tanh verhindert extreme Werte!

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# model erstellen
input_dim = data.shape[1]
latent_dim = 10
vae = VAE(input_dim, latent_dim)
optimizer = optim.Adam(vae.parameters(), lr=0.001)

def loss_function(recon_x, x, mu, logvar):
    recon_loss = nn.SmoothL1Loss()(recon_x, x)  # besser als MSE für Zeitreihen
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + 0.001 * kl_loss  # KL-Loss gewichtet reduzieren

# training mit progress-monitoring
epochs = 50
vae.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(batch)
        loss = loss_function(recon_batch, batch, mu, logvar)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

vae.eval()

# sampling aus dem trainierten latenten Raum
with torch.no_grad():
    real_mu = []
    for batch in dataloader:
        mu, _ = vae.encode(batch)
        real_mu.append(mu)
    real_mu = torch.cat(real_mu, dim=0)  # latente Verteilung aus echten Daten!

# sampling basierend auf realen latenten Codes
z_sample = real_mu + 0.1 * torch.randn_like(real_mu)  # kleines Rauschen hinzufügen
synthetic_profiles = vae.decode(z_sample).detach().numpy()

synthetic_profiles = synthetic_profiles * std + mean

# formatanpassung
synthetic_profiles = synthetic_profiles[:data.shape[0], :]
synthetic_df = pd.DataFrame(synthetic_profiles, columns=df.columns[1:])
synthetic_df.insert(0, "timestamp", timestamps.values[:synthetic_df.shape[0]])

# speichern der synthetischen Daten
output_path = Path.cwd() / "synthetic_smart_meter_data.csv"
synthetic_df.to_csv(output_path, index=False)

print(f"Synthetische Daten gespeichert: {output_path}")
